In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
class LogisRegression:
  def __init__(self, learning_rate, n_iters):
    self.lr = learning_rate
    self.n_iters = n_iters
    self.weights = None
    self.bias = None


  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))    # 1 / (1 + np.exp(-x))

  def compute_loss(self, _y, y_pred):

    # _y = np.array(_y)  # Convert _y to a NumPy array
    # y_pred = np.array(y_pred)  # Convert y_pred to a NumPy array

    # Ensure inputs are Pandas Series or DataFrames
    if isinstance(_y, pd.DataFrame) or isinstance(_y, pd.Series):
        _y = _y.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values

    # Avoid division by zero in log function
    _y = np.clip(_y, 1e-10, 1 - 1e-10)
    y_pred = np.clip(y_pred, 1e-10, 1 - 1e-10)

    loss = (-_y * np.log(y_pred) - (1 - _y) * np.log(1 - y_pred)).mean()
    return loss

  def fit(self, X, y):
    # init parameters
    n_samples, n_features = X.shape
    self.weights = np.zeros(n_features)
    self.bias = 0


    tolerance = 1e-6  # Tolerance for convergence
    previous_loss = float('inf')

    # gradient descent
    for i in range(self.n_iters):

      y_pred = self.predict(X)

      dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
      db = (1 / n_samples) * np.sum(y_pred - y)

      self.weights -= self.lr * dw
      self.bias -= self.lr * db

      current_loss = self.compute_loss(y, y_pred)


      if abs(previous_loss - current_loss) < tolerance:
        print(f"Converged at iteration {i}")
        break
      previous_loss = current_loss

  def predict(self, X):
    model = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(model)
    y_pred_cls = [1 if i > 0.5 else 0 for i in y_pred]
    return y_pred_cls




In [3]:
df = pd.read_csv('/content/drive/MyDrive/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
# check for missing values
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [6]:
# check for duplicate values
df.duplicated().sum()

0

In [7]:
# check for info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [8]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
mle = LogisRegression(learning_rate=0.001, n_iters=1000)
mle.fit(X_train, y_train)
y_pred = mle.predict(X_test)

Converged at iteration 11


In [11]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Accuracy: 0.35714285714285715
Confusion Matrix:
 [[ 0 99]
 [ 0 55]]
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        99
           1       0.36      1.00      0.53        55

    accuracy                           0.36       154
   macro avg       0.18      0.50      0.26       154
weighted avg       0.13      0.36      0.19       154

